In [1]:
import numpy as np
import torch 
from terratorch import BACKBONE_REGISTRY



c:\Users\timvd\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.21 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


ImportError: cannot import name 'BACKBONE_REGISTRY' from 'terratorch' (c:\Users\timvd\AppData\Local\Programs\Python\Python311\Lib\site-packages\terratorch\__init__.py)